In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 

# Legacy seismology: Digitizing old waveforms for modern seismology
## IASPEI Early Career Scientific School on digitizing legacy seismic data and validating them using ocean microseism

## Authors: 
* Raphael De Plaen ([@RDePlaen](https://github.com/RDePlaen))
* Thomas Lecocq ([@seismotom](https://github.com/ThomasLecocq))
* Josep Batllo

## Introduction:

Analog seismograms on photographic paper, smoked paper and other magnetic support are records of ground motion that cover most of the 20th century. Whether for past earthquakes or the analysis of continuous wavefiled for time dependent seismology, this represents a wealth of information that can not be found anywhere else but remains hard to exploit with modern tools.
In this excersise, we will explore the digitisation of old seismic records and the challenges that can be encountered before they can be used for modern seismology.
The paper seismograms that we will use have already been scanned but need to be vectorised.

Throughout much of the 20th century, ground motion was recorded on analog seismograms using photographic paper, smoked paper, magnetic media, etc. 
These records capture an invaluable archive of past earthquakes and continuous seismic signals, offering insights that are unavailable from modern datasets alone, seismic or otherwise. 
Making use of this information today is challenging, as the original formats are not directly compatible with modern analysis tools.

In this exercise, we will explore the process of digitizing historical seismic records and discuss the main challenges involved in preparing them for contemporary seismological studies. The seismograms we will work with have already been scanned, but they still need to be converted into vectorized form before they can be analyzed.

<p align="center">
  <img src="./figures/gal.jpg" width=800></img>
</p>

This morning's exercise will teach you:
1. How to vectorize seismic traces from a scanned seismogram. 
2. How to convert the time series into ground motion using the transfer function
3. Validate the spectra of the vectorised trace by comparing it with corresponding WaveWatch III ocean model

## References:
* Lecocq, T., Ardhuin, F., Collin, F., & Camelbeeck, T. (2020). On the extraction of microseismic ground motion from analog seismograms for the validation of ocean‐climate models. Seismological Research Letters, 91(3), 1518-1530.

* Ishii, M., & Ishii, H. (2022). DigitSeis: software to extract time series from analogue seismograms. Progress in Earth and Planetary Science, 9(1), 50.

* Tomasetto, L., Boué, P., Ardhuin, F., Stutzmann, É., Xu, Z., de Plaen, R., & Stehly, L. (2025, April). WMSAN: Wave Model Sources of Ambient Noise Python Library. From Modeling to Applications. In EGU General Assembly-Ambient Seismic Noise and Seismic Interferometry.

In [ ]:
%matplotlib inline
import os
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from obspy import Trace, Stream, UTCDateTime
from matplotlib.mlab import psd, magnitude_spectrum
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage import feature
import matplotlib.image as mpimg
import matplotlib.dates as mdatesAnalog seismograms on photographic paper, smoked paper and other magnetic support are records of ground motion that cover most of the 20th century. Whether for past earthquakes or the analysis of continuous wavefiled for time dependent seismology, this represents a wealth of information that can not be found anywhere else but remains hard to exploit with modern tools.


import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import FormatStrFormatter
from matplotlib.gridspec import GridSpec

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from skimage.morphology import skeletonize
from skimage.transform import rotate, hough_line, hough_line_peaks
import scipy.signal as ss
import seaborn as sns

from PIL import Image
import PIL.ImageOps  

from obspy.signal.spectral_estimation import get_nlnm, get_nhnm

import matplotlib as mpl
mpl.rcParams['date.autoformatter.day'] = "%Y-%m-%d"
mpl.rcParams['date.autoformatter.hour'] = "%Y-%m-%d %Hh"
mpl.rcParams['date.autoformatter.minute'] = "%Y-%m-%d %H:%M"

mpl.rcParams['figure.figsize'] = "12,8"
mpl.rcParams['figure.dpi'] = 100

mpl.rcParams['axes.axisbelow'] = True


# Exercise 2:
## Unsupervised vectorisation of continuous seismic records

In the first exercise, we explored the vectorisation os seismic waveforms using [DigitSeis](references/digitseis_2022.pdf), a supervised digitisation tool. While it is efficient to solve complicated overlaping traces from earthquakes and other high implitude events, you also experienced how time consuming this approach can be. Since an increasing amount of modern methods rely on seismic ambient noise, an alternative approach is the extract the background traces and ingnore the complicated overlaping cases. While less comprehensive, this approach is also a lot faster and does not require supervision.

For this exercises, we are reproducing the vectorization of the Galitzin recording of the "Big Flood",  a catastrophic flood caused by a heavy storm surge that severly impacted the Netherlands, Belgium and the UK in 1953, as detailed in [Lecocq et al. (2020)](references/lecocq_2020_srl.pdf)
. The waveforms were recoded in Brussels and emphasize, the importance of studying the temporal variation of oceanic microseism in Legacy seismic data to better understand the Oceanic climate of the 20th century.

| ![](./figures/flood_1953.jpg) | ![](./figures/Galitzin_horiz.jpg) |
|-----------------|-----------------|
| Aftermath of the "Big Flood" in the Dutch Islands | The horizontal Galitzin seismometer of The Royal Observatory of Belgium in Uccle |




In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])


In [ ]:
traces = {}

In [ ]:
# Loading the scanned file

files = sorted(glob.glob(r"./data/19530*.jpg"))

PLOT = True
# Galitzin: bottom to up, left to right

for file in files[1:]:
    print("Processing %s"%file)
    filename = os.path.split(file)[1]
    if filename in traces:
        continue

    im = Image.open(file)
#     im = im.rotate(90)
    im = PIL.ImageOps.invert(im)

    im = np.asarray(im).copy()
    print(im.shape)
    if len(im.shape) > 2:
        im = rgb2gray(im) 

    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(im, aspect='auto', interpolation="none")
        plt.savefig("figures/image_0_raw.jpg")
    # apply threshold
    image = im.copy()
    thresh = threshold_otsu(image)
    bw = closing(image > thresh, square(3))
    
    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(bw, aspect='auto', interpolation="none")
        plt.savefig("figures/image_2_bw.jpg")
    
    
    # remove artifacts connected to image border
    cleared = clear_border(bw)
    
    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(cleared, aspect='auto', interpolation="none")
        plt.savefig("figures/image_3_cleared.jpg")
    
    angles = np.deg2rad(np.linspace(85,95,200))
    hspace, angles, dists = hough_line(cleared, theta=angles)
    hspace, angles, dists = hough_line_peaks(hspace, angles, dists)
    angle = np.median(np.rad2deg(angles)) - 90
    cleared = rotate(cleared, angle)
    
    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(cleared, aspect='auto', interpolation="none")
        plt.savefig("figures/image_4_rotated.jpg")
    
    
    print(cleared.shape)
    # label image regions
    label_image = label(cleared, connectivity=2)
    print(label_image.shape)
    image_label_overlay = label2rgb(label_image, image=cleared, bg_label=0, alpha=0.8)
    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(image_label_overlay, aspect='auto', interpolation="none")
        plt.savefig("figures/image_5_label_overlay.jpg")
    i = 0
    all = []
    all_s = {}
    all_box = []
    all_id = {}
    for region in regionprops(label_image):
        # take regions with large enough area
        if region.area >= 100:
            # check zone size, if too small on time axis or too wide on amplitude, reject
            _minr, _minc, _maxr, _maxc = region.bbox
            if _maxc-_minc <= 50:
                continue
            if _maxr - _minr >= 100:
                continue
            # draw rectangle around segmented traces
            minr, minc, maxr, maxc = region.bbox
            if PLOT:
                rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                          fill=False, edgecolor='red', linewidth=0.1)
                plt.text((maxc+minc)/2, minr, "%05i" %i, color='w', fontsize=2, horizontalalignment="center")
                ax.add_patch(rect)
            i+=1
            tmp = cleared[minr:maxr,minc:maxc]
            tmp ^= tmp.min()
            tmp[tmp>0] = 1.0
            tmp = np.ascontiguousarray(tmp, dtype=np.uint8)
            skeleton = skeletonize(tmp)
            xx = []
            for _ in range(skeleton.shape[1]):
                _ = skeleton[:,_]
                __ = np.where(_==1)[0]
                if len(__):
                    xx.append(__[0])
                else:
                    if len(xx):
                        xx.append(xx[-1])
                    else:
                        xx.append(0)
            _ = np.asarray(xx, dtype='float')
            if _[0] == 0:
                _[0] = _[1]

            if PLOT:
                plt.plot(np.arange(minc,maxc),minr+_, c='w', lw=0.1, zorder=100)
            _ -= _.mean()
            trace = {"x": minc, "y":(minr+maxr)/2., "data":_ }
            all.append(trace)

    traces[filename] = all
    if PLOT:
        ax.set_axis_off()
        plt.tight_layout()
        plt.savefig("figures/image_6_skeleton.jpg")
#         plt.show()
    del image, im, label_image, bw, cleared, image_label_overlay
    break
print("Done")
plt.show()

In [ ]:
day = list(traces.keys())[0].split('.')[0]

The start time for each file/day has been collected from the scanned seismogram

In [ ]:
start_times = {}
start_times["19530115.jpg"] = "07:30:00"
start_times["19530116.jpg"] = "07:30:00"
start_times["19530117.jpg"] = "07:22:00"
start_times["19530118.jpg"] = "09:40:00"
start_times["19530119.jpg"] = "06:59:00"
start_times["19530120.jpg"] = "07:02:00"
start_times["19530121.jpg"] = "07:52:00"
start_times["19530122.jpg"] = "07:30:00"
start_times["19530123.jpg"] = "07:25:00"
start_times["19530124.jpg"] = "07:08:00"
start_times["19530125.jpg"] = "09:50:00"
start_times["19530126.jpg"] = "07:10:00"
start_times["19530127.jpg"] = "07:08:00"
start_times["19530128.jpg"] = "07:45:00"
start_times["19530129.jpg"] = "07:43:00"
start_times["19530130.jpg"] = "07:25:00"
start_times["19530131.jpg"] = "07:30:00"
start_times["19530201.jpg"] = "09:45:00"
start_times["19530202.jpg"] = "07:30:00"
start_times["19530203.jpg"] = "07:19:00"
start_times["19530204.jpg"] = "07:54:00"
start_times["19530205.jpg"] = "07:58:00"
start_times["19530206.jpg"] = "08:00:00"
start_times["19530207.jpg"] = "07:30:00"
start_times["19530208.jpg"] = "09:14:00"
start_times["19530209.jpg"] = "09:27:00"
start_times["19530210.jpg"] = "07:47:00"
start_times["19530211.jpg"] = "08:08:00"
start_times["19530212.jpg"] = "07:00:00"
start_times["19530213.jpg"] = "08:15:00"
start_times["19530214.jpg"] = "08:00:00"
start_times["19530215.jpg"] = "10:14:00"

In [ ]:
processed_traces = []


for key in sorted(traces):
    print(key)
    date,_ = key.split('.')
    d = datetime.datetime.strptime(date+ " %s" % start_times[key], "%Y%m%d %H:%M:%S")
    df = pd.DataFrame(traces[key].copy())
    df = df.sort_values(["y","x"], ascending=[True,True])
    ppmin = np.median([len(a) for a in df["data"]])
    print("ppmin", ppmin)
    if (ppmin < 300 or ppmin > 400) and key != "19530118.jpg":
        ppmin=351
    pps = ppmin / 59.0
    timer = 0
    for id, row in df.iterrows():
        tr = Trace(data=row["data"].copy())
        tr.data *= 0.0254/300.0 # data is now in meters
        if id == 10:
            print(tr.data.ptp())
        tr.stats.sampling_rate = pps
        tr.stats.starttime = UTCDateTime(d) + timer
        tr.detrend("linear")
        tr.interpolate(8, method='lanczos', a=64)
        tr.taper(None, max_length=0.5, side="both")
        tr.filter("highpass", freq=0.05, corners=8)
        if id == 10:
            print(tr.data.ptp())
        processed_traces.append(tr)
        timer += tr.stats.npts*tr.stats.delta + 1


In [ ]:
st = Stream(traces=processed_traces)
st.write("{}.BHZ.mseed".format(day))


# Waveform restitution

The amplitude response is calculated from the constants collected from the 1953 seismic bulletin of the Royal Observatory of Belgium.
The response plot highlight how the Galitzin instruments were particularly sensitive to the Primary and Secondary microseisms.

In [ ]:
def restitution_galitzin(ym, Tp=None, kind='horizontal', plot=False, show=True):
    if Tp is None:
        Tp = np.logspace(-1,2,num=200)
    if kind == "horizontal":

        T = 21.5
        T1 = 21.8
        l = 124.7
        mu = 0.2
        A1 = 1040.0
        k = 38.0
    
    elif kind == "vertical":
        T = 10.0
        T1 = 10.15
        l = 173.8 # somville 1937
        mu = 0.0
        A1 = 1060.0 #Somville 1930 bulletin
        k = 290.0 # somville 1953  bulletin + correction for mistake in parameters leading to unrealistic magnitudes

    else:
        print("kind must be either horizontal or vertical, quitting")
        return False

    u = Tp / T
    u1 = Tp / T1

    fu = (2.*u/(1.+u**2))**2
    C1 = np.pi * l / (k*A1)
    xm = C1 * ym/Tp * (1.+u**2)  * (1.+u1**2)  * np.sqrt(1.-fu*mu**2)
    
    if plot:
        
#         plt.title("ROB Galitzin Seismometer Amplitude Response (%s)" % )
        base, = plt.loglog(Tp,1./xm, lw=2)
        maxx = np.argmax(1./xm)
        plt.axvline(Tp[maxx], c=base.get_color(), ls="--", 
                    label='%s Maximum: %.1f at %.1f s'%(kind.capitalize(), 1./xm[maxx], Tp[maxx]))
        plt.grid(which='both')
        plt.xlabel("Period (s)")
        plt.ylabel("Amplification")
        plt.legend(loc=2)
        if show:
            plt.show()
    return xm

In [ ]:
# HORIZONTAL GALITZIN ROB
ym = 1.0
Tp = np.logspace(np.log10(0.2),2,num=200)

plt.figure(figsize=(12,5))
xm = restitution_galitzin(ym, Tp, kind="horizontal", plot=True, show=False)
xm = restitution_galitzin(ym, Tp, kind="vertical", plot=True, show=False)
plt.grid(which="both")
plt.tight_layout()
plt.savefig("figures/Amplitude_response_galitzin.png")
plt.show()

Now let's load the whole period

In [ ]:
from obspy import read
processed_traces=read('./data/1953.BHZ.mseed')

In [ ]:
processed_traces

In [ ]:
all_x = []
all_pxx = []
all_amp = []
all_df2 = []
all_times = []

all_ptp = []
all_domP = []

for tr in processed_traces:
    _ = tr.data.copy()
    pxx, freqs = psd(tr.data, Fs=tr.stats.sampling_rate, pad_to=1024, detrend="mean", noverlap=8, scale_by_freq=True)
    all_ptp.append(np.max(np.abs(tr.data)))
    maxP = np.min([12., 1./freqs[np.argmax(pxx)]])
    amp =  restitution_galitzin(_, maxP, kind="vertical", plot=False)
    pxx, freqs = psd(amp, Fs=tr.stats.sampling_rate, pad_to=1024, detrend="mean", noverlap=8, scale_by_freq=True)
    all_domP.append(maxP)
    all_x.append(freqs)
    all_pxx.append(pxx)
    freqs[0] = freqs[1]*0.1
    all_amp.append(pxx)
    all_times.append(tr.stats.starttime.datetime)
df2 = pd.DataFrame(all_amp, columns=all_x[0], index=all_times)


In [ ]:
df2.head()

In [ ]:
amp =  restitution_galitzin(np.ones(len(all_domP)),np.array(all_domP), kind="vertical", plot=False)
s = pd.Series(np.array(all_ptp)*amp*1e6, index=pd.DatetimeIndex(all_times))
print(amp)
plt.figure()
plt.scatter(s.index, s )
r = s.resample("30min").quantile(0.95)
plt.plot(r.index, r , c='blue')
r = s.resample("30min").mean()
plt.plot(r.index, r , c='purple')
plt.ylabel("Displacement, µm")
plt.show()


Resample to match the ocean model

In [ ]:
from scipy.stats import trim_mean, tmean, scoreatpercentile
df3 = df2.copy()
df3 = df3.loc[:,1.0/18:]

rs = df3.resample("3H", axis=0)
df3 = rs.quantile(0.95)

df3.head()

In [ ]:

periods = 1. / df3.columns

cmap = plt.get_cmap('inferno')
cmap.set_bad(color = 'k', alpha = .6)

# print(len(pmax), len(df3.index))
fig = plt.figure(figsize=(10,4))


pmax = []
for d in df3.values:
    if np.any(np.isnan(d)):
        pmax.append(np.nan)
    else:
        pmax.append(periods[np.argmax(d)])
data = df3.values.T

print(np.any(np.isnan(data)))
plt.pcolormesh( df3.index.to_pydatetime().ravel(), periods, data, cmap=cmap, rasterized=True)
plt.colorbar()
plt.ylabel("Period (s)")
plt.xlabel("Date")
plt.ylim(0.2,15)
fig.autofmt_xdate()
plt.tight_layout()
plt.savefig("figures/1953.png")
plt.show()

# Modelling Ocean

The ocean model are generated using Wavewatch III and used specifically to simulate the corresponding oceanic microseisms. 
These sources are then propagated to the selected seismic station to simulate the corresponding ground motion. To explore more on this subject, see [Tomasetto et al. (2024)](references/tomasetto_2024.pdf).

In [ ]:
import netCDF4
ncname = r"./model/UCC_195301-195311_Q0200REF102040_edzf.nc"

nc = netCDF4.Dataset(ncname)
edzf = nc.variables["edzf"]
print(edzf)
print(edzf.units)
times = nc.variables['time']
freqsnc = nc.variables['frequency']
jd = netCDF4.num2date(times[:],times.units)
model = pd.DataFrame(edzf[:,0,:],index=jd, columns=freqsnc)
model.head()

# Comparing the spectra of the digitized waveform with the ocean model

In [ ]:
model.index = pd.to_datetime([t.strftime("%Y-%m-%d %H:%M:%S") for t in model.index])
day = model.loc["1953-01-15":"1953-02-15"].copy()
day = day.resample("3H").median()

fig = plt.figure()
ax = plt.subplot(211)
periods = 1. / df3.columns

cmap = plt.get_cmap('inferno')
cmap.set_bad(color = 'k', alpha = .6)

observed = df3.copy()

pmax = []
for d in observed.values:
    if np.any(np.isnan(d)):
        pmax.append(np.nan)
    else:
        pmax.append(periods[np.argmax(d)])

# for id, row in observed.iterrows():
#     observed.loc[id] /= ynew**3
        
data = 10* np.log10(observed.values).T
# data = observed.values.T
# data = np.sqrt(data)

# print(len(observed.columns), data.shape, data.shape[1], ynew.shape)


plt.pcolormesh( observed.index.to_pydatetime().ravel(), periods, data, cmap=cmap,  rasterized=True, vmax=-100, vmin=-140)
cb = plt.colorbar(pad=0.01)
cb.set_label('Decibel (dB)')

# plt.semilogy(df3.index.to_pydatetime().ravel(), pmax, "o", c="w")
# plt.semilogy(df3.index[0], 1.0 )
plt.ylim(1, 50)
plt.ylabel("Period (s) | DPSD (dB)")
plt.xlabel("Date")
plt.ylim(0.2,20)
# fig.autofmt_xdate()
plt.tight_layout()
plt.title("Data")
plt.xlabel("")
plt.subplot(212, sharex=ax, sharey=ax)

data = np.asarray(day)

# for i,d in enumerate(data):
#     data[i] *= (2*np.pi/day.columns)**2
data = 10* np.log10(data).T
# data *= 1e3
# data = np.sqrt(data)
plt.pcolormesh(day.index, 1./day.columns, data, cmap=cmap, vmax=-100, vmin=-140)
cb = plt.colorbar(pad=0.01)
cb.set_label('Decibel (dB)')


# plt.semilogy(df3.index.to_pydatetime().ravel(), pmax, "o", c="w")
# plt.semilogy(df3.index[0], 1.0 )
plt.ylabel("Period (s) | DPSD (dB)")
plt.xlabel("Date")
plt.title("Model")
plt.ylim(2,12)
# fig.autofmt_xdate()
plt.xlabel("")

plt.tight_layout()


# plt.subplots_adjust(hspace=0.02)
plt.savefig("1953_observed_vs_%s.pdf"%os.path.split(ncname)[1])
plt.savefig("1953observedvsmodel_spectrum.png",bbox_inches='tight')
plt.show()

In [ ]:
plt.figure()

data = 10* np.log10(observed.values).T
print(data.shape)

plt.plot(periods, np.nanmean(data, axis=1), label="Jan-Feb 1953 - V - Data")

data = np.asarray(day)

data = 10* np.log10(data).T

plt.plot(1./day.columns, np.nanmean(data, axis=1), label="Jan-Feb 1953 - V - Model")

p, a = get_nlnm()
plt.plot(p, a)
p, a = get_nhnm()
plt.semilogx(p, a)
plt.legend()
plt.xlim(0.5,50)
plt.show()

In [ ]:
T1 = df3.copy()#.dropna(axis="index", how="all").dropna(axis="columns", how='all')
T2 = day.copy()#.dropna(axis="index", how="all").dropna(axis="columns", how='all')

from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)

idx = T1.index.intersection(T2.index)

tmp = T1.loc[idx].copy()
tmp2 = T2.loc[idx].copy()

# df = 1./8.0
# print(df)
# def psd_to_amp(a):
#     return np.sqrt(a * df * 3)

# tmp = tmp.apply(psd_to_amp)
# tmp2 = tmp2.apply(psd_to_amp)

def rms(a):
    sel = a.loc[0.08:0.32]
#     print(sel.index)
#     sel -= sel.min()
    return np.sqrt(np.trapz(sel.values, sel.index))*1.0e6

test = tmp.apply(rms, axis=1)
test2 = tmp2.apply(rms, axis=1)

tmp = tmp.apply(np.sqrt)
tmp2 = tmp2.apply(np.sqrt)


conv = test2.max()/test.max()
print("Conversion Factor: %f"%conv)
# print(conv, 1./conv)

datamax = tmp.max(axis=1)
def minmax(frame, dmax):
    
    low = []
    high= []
    
    idx = frame[frame>=dmax]
    return idx
        
datamax95 = tmp.apply(minmax, dmax=datamax*0.70)


sns.set_palette("Dark2")

plt.figure(figsize=(12,6))
ax1 = plt.subplot(211)
# plt.plot(tmp.index, tmp.max(axis=1)*conv, label="data * %g"%conv)
# plt.plot(tmp2.index, tmp2.max(axis=1), c='r', label="model")

plt.plot(test.index, test, '-+', label="data", markersize=5)
# plt.plot(test.index, test*conv, '-+', label="data scaled", markersize=5)
plt.plot(test2.index, test2, label="model")

dr = pd.date_range(test.index[0].date(), test.index[-1].date(), freq="D")
drl = [d.strftime("%Y-%m-%d") for d in dr]
plt.xticks(dr, drl)

plt.legend(loc=2)
plt.ylabel("$\delta_{rms}$ (µm)")
plt.grid(True)

# plt.twinx()
# localsource = pd.read_csv("localsources.csv", index_col=0, parse_dates=True).loc["1953-01-15":"1953-02-15"]
# plt.plot(localsource.index, localsource, c='k')


# 


# a = 1./tmp2.idxmax(axis=1)
# b = 1./tmp.idxmax(axis=1)

# plt.plot([3.,10.], [3.,10.], c="r")
# plt.xlim(3,10)
# plt.ylim(3,10)

# sns.regplot(x=a, y=b)


# plt.xticks(np.arange(3,11),np.arange(3,11), zorder=-10)
# plt.yticks(np.arange(3,11),np.arange(3,11), zorder=-10)
# plt.xlabel("Model (s)")
# plt.ylabel("Data (s)")
# plt.title("Max Peak Period")


ax3 = plt.subplot(212, sharex=ax1)
datamax = tmp.idxmax(axis=1)
plt.plot(tmp.index, 1./datamax, "-+", label="data", markersize=5)
plt.xticks(dr, drl)
indexes = []
mins = []
maxs = []
for index, row in datamax95.iterrows():
    if np.all(np.isnan(row)):
        continue
    else:
        row.dropna(inplace=True)
        indexes.append(index)
        mins.append(1./row.first_valid_index())
        maxs.append(1./row.last_valid_index())

plt.fill_between(indexes, mins, maxs, zorder=-1, facecolor='lightblue', alpha=0.3)


plt.plot(tmp2.index, 1./tmp2.idxmax(axis=1), label="model")

plt.xlim(tmp2.index[0], tmp2.index[-1])

plt.legend(loc=2)
plt.ylim(3,10)
plt.ylabel("Period of PSD max value (s)")
plt.grid(True)
plt.gcf().autofmt_xdate()


a = pd.concat((test, test2), axis=1)




ax2 = plt.axes([0,0,1,1], aspect='equal', zorder=100)
ip = InsetPosition(ax1, [0.01,0.60,0.35, 0.35])
ax2.set_axes_locator(ip)
plt.sca(ax2)
plt.scatter(a[0], a[1], s=2, c='k')
plt.plot([0,2], [0,2],c='k')
sns.regplot(x=a[0], y=a[1], ax=ax2, scatter=False, label=False)
plt.xlabel('data')
plt.ylabel('model')
plt.xlim(0,2)
plt.ylim(0,2)



ax4 = plt.axes([0,0,.99,.99], aspect='equal', zorder=100)
ip2 = InsetPosition(ax3, [0.01,0.60,0.35, 0.35])
ax4.set_axes_locator(ip2)
plt.sca(ax4)
a = 1./tmp.idxmax(axis=1)
b = 1./tmp2.idxmax(axis=1)
plt.scatter(a, b, s=2, c='k')
plt.plot([0,9], [0,9],c='k')
sns.regplot(x=a, y=a, ax=ax4, scatter=False, label=False)
plt.xlabel('data')
plt.ylabel('model')
plt.xlim(4,9)
plt.ylim(4,9)



plt.subplots_adjust(hspace=0.02)
plt.tight_layout()
# plt.grid(True)
# plt.savefig("maxvalueandperiod.pdf")
plt.savefig("figures/1953observedvsmodel_valper_Z.png")
plt.show()
